In [23]:
import numpy as np
import pandas as pd

In [129]:
with pd.ExcelFile('data/Sales Department Input.xlsx') as xl:
    df = pd.concat([pd.read_excel(xl, s).assign(sheet=s) for s in xl.sheet_names])\
        .rename(columns={'Unnamed: 7': 'YTD Total'})

df['month'] = df['Date'].astype('datetime64[M]').fillna(method='ffill')
df['year'] = df['Date'].astype('datetime64[Y]').fillna(method='ffill')

# Fill in the Salesperson names for each row (the name appears at the bottom of each monthly grouping)
df['Salesperson'] = df['Salesperson'].fillna(method='bfill')

# Reshape the data so all the bike types are in a single column
df_m =df[df['Date'].notna()].drop(columns=['RowID', 'Total', 'YTD Total', 'sheet'])\
        .melt(id_vars=['Salesperson', 'Date', 'month', 'year'],
             var_name='Bike Type', value_name='Sales')

# Bring out the YTD information from the October tracker and use it to create YTD totals for November too
df_tot = df_m.groupby(['month', 'year', 'Salesperson'])['Sales'].sum().reset_index()\
        .merge(df[df['YTD Total'].notna()][['Salesperson', 'YTD Total', 'month']],
             how='left', on=['Salesperson', 'month'])\
        .sort_values(by=['Salesperson', 'month'])

df_tot['YTD Total2'] = np.where(df_tot['YTD Total'].isnull(), df_tot['Sales'], df_tot['YTD Total'])
df_tot['YTD Total'] = df_tot.groupby(['Salesperson', 'year'])['YTD Total2'].cumsum()

df_tot.drop(columns=['Sales', 'YTD Total2'], inplace=True)
df_m = df_m.merge(df_tot, how='left', on=['month', 'Salesperson'])

# output
df_m= df_m[['Salesperson', 'Date', 'Bike Type', 'Sales', 'YTD Total']]
df_m.to_csv('output-2021-week50.csv', index=False, date_format='%d/%m/%Y')

In [113]:
df['C'] = df['A'].cumsum()
df

,A,B,C
0,2.0,1.0,2.0
1,3.0,NaN,5.0
2,1.0,0.0,6.0
